# 🔥 지역난방 열수요 예측: 시즌별-브랜치별 LightGBM 모델

## 📋 모델링 전략
- **시즌 분할**: Heating Season vs Non-Heating Season
- **브랜치별 개별 모델**: 각 branch_id마다 전용 모델
- **LightGBM**: 모든 모델에 LightGBM 사용
- **하이퍼파라미터 최적화**: Optuna TPE 사용
- **총 모델 수**: 38개 (2시즌 × 19브랜치)

In [16]:
# Google Colab 환경 확인 및 패키지 설치
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🔥 Google Colab 환경에서 실행 중...")
    !pip install xgboost optuna
    from google.colab import files, drive
    print("✅ 패키지 설치 완료!")
else:
    print("💻 로컬 환경에서 실행 중...")

💻 로컬 환경에서 실행 중...


In [17]:
# 1. 라이브러리 import 수정
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from tqdm.auto import tqdm

# TensorFlow/Keras (LSTM용)
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

# 기존 머신러닝
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Optuna
import optuna
from optuna.samplers import TPESampler

# GPU 설정
print(f"🚀 TensorFlow 버전: {tf.__version__}")
if tf.config.list_physical_devices('GPU'):
    print(f"✅ GPU 사용 가능: {tf.config.list_physical_devices('GPU')}")
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        tf.config.experimental.set_memory_growth(gpus[0], True)
else:
    print("💻 CPU 사용")

🚀 TensorFlow 버전: 2.19.0
💻 CPU 사용


In [18]:
# 데이터 파일 로드
if IN_COLAB:
    print("📁 파일 업로드 방법 선택:")
    print("1. 직접 업로드")
    print("2. Google Drive")

    method = input("선택 (1 또는 2): ")

    if method == "1":
        uploaded = files.upload()
        files_list = list(uploaded.keys())
        train_path = [f for f in files_list if 'train' in f.lower()][0]
        test_path = [f for f in files_list if 'test' in f.lower()][0]
    else:
        drive.mount('/content/drive')
        train_path = "/content/drive/MyDrive/train_heat.csv"
        test_path = "/content/drive/MyDrive/test_heat.csv"
else:
    train_path = 'dataset/train_data_2122.csv'
    test_path = 'dataset/test_data_23.csv'

print(f"✅ 파일 경로 설정 완료")

✅ 파일 경로 설정 완료


## 1️⃣ 데이터 로드 및 전처리

In [19]:
def load_and_preprocess(train_path, test_path):
    print("📊 데이터 로드 및 전처리...")

    # 데이터 로드
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)

    def process_df(df):
        # 컬럼명 정리
        if 'Unnamed: 0' in df.columns:
            df = df.drop(columns=['Unnamed: 0'])
        df.columns = [col.replace('train_heat.', '') for col in df.columns]

        # 시간 변수
        df['datetime'] = pd.to_datetime(df['tm'], format='%Y%m%d%H')
        # df['year'] = df['datetime'].dt.year
        df['month'] = df['datetime'].dt.month
        # df['day'] = df['datetime'].dt.day
        df['hour'] = df['datetime'].dt.hour
        df['dayofweek'] = df['datetime'].dt.dayofweek

        # 결측치 처리
        missing_cols = ['ta', 'wd', 'ws', 'rn_day', 'rn_hr1', 'hm', 'si', 'ta_chi']
        if 'heat_demand' in df.columns:
            missing_cols.append('heat_demand')

        for col in missing_cols:
            if col in df.columns:
                df[col] = df[col].replace(-99, np.nan)

        # wd -9.9 결측치 처리
        df.loc[df['wd'] == -9.9, 'wd'] = np.nan

        # 일사량 야간 처리
        if 'si' in df.columns:
            night_mask = (df['hour'] < 8) | (df['hour'] > 18)
            df.loc[night_mask & df['si'].isna(), 'si'] = 0

        # 지사별 보간
        df = df.sort_values(['branch_id', 'datetime'])
        numeric_cols = df.select_dtypes(include=[np.number]).columns

        for branch in df['branch_id'].unique():
            mask = df['branch_id'] == branch
            df.loc[mask, numeric_cols] = df.loc[mask, numeric_cols].interpolate().fillna(method='ffill').fillna(method='bfill')

        return df

    train_df = process_df(train_df)
    test_df = process_df(test_df)

    print(f"   훈련: {train_df.shape}, 테스트: {test_df.shape}")
    print(f"   기간: {train_df['datetime'].min()} ~ {test_df['datetime'].max()}")
    print(f"   브랜치: {sorted(train_df['branch_id'].unique())}")

    return train_df, test_df

train_df, test_df = load_and_preprocess(train_path, test_path)

📊 데이터 로드 및 전처리...
   훈련: (332861, 15), 테스트: (166440, 15)
   기간: 2021-01-01 01:00:00 ~ 2023-12-31 23:00:00
   브랜치: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S']


## 2️⃣ 파생변수 생성

In [20]:
def create_features(df):
    """HDD, wind_chill, 순환형 인코딩, 범주형 변수 생성"""
    df = df.copy()

    # ⭐ HDD (수치형)
    if 'ta' in df.columns:
        df['HDD_18'] = np.maximum(18 - df['ta'], 0)
        df['HDD_20'] = np.maximum(20 - df['ta'], 0)

    # ⭐ wind_chill (수치형)
    if 'ta' in df.columns and 'ws' in df.columns:
        df['wind_chill'] = np.where(
            (df['ta'] <= 10) & (df['ws'] > 0),
            13.12 + 0.6215 * df['ta'] - 11.37 * (df['ws'] ** 0.16) + 0.3965 * df['ta'] * (df['ws'] ** 0.16),
            df['ta']
        )

    # ⭐ heating_season (범주형)
    df['heating_season'] = df['month'].isin([10, 11, 12, 1, 2, 3, 4]).astype(int)

    # 시간대 범주형
    df['is_weekend'] = (df['dayofweek'] >= 5).astype(int)
    df['is_peak_morning'] = ((df['hour'] >= 7) & (df['hour'] <= 9)).astype(int)
    df['is_peak_evening'] = ((df['hour'] >= 18) & (df['hour'] <= 22)).astype(int)
    df['is_night'] = ((df['hour'] >= 23) | (df['hour'] <= 5)).astype(int)

    # 피크시간 통합
    df['peak_time_category'] = 0
    df.loc[df['is_peak_morning'] == 1, 'peak_time_category'] = 1
    df.loc[df['is_peak_evening'] == 1, 'peak_time_category'] = 2
    df.loc[df['is_night'] == 1, 'peak_time_category'] = 3

    # ⭐ 기온 범주 (범주형)
    if 'ta' in df.columns:
        df['temp_category'] = pd.cut(df['ta'],
                                   bins=[-np.inf, 0, 10, 20, 30, np.inf],
                                   labels=[0, 1, 2, 3, 4]).astype(int)

    # ⭐ 강수 강도 (범주형)
    if 'rn_day' in df.columns:
        df['rain_intensity'] = pd.cut(df['rn_day'],
                                   bins=[-1, 0, 1, 5, 10, np.inf],
                                   labels=[0, 1, 2, 3, 4]).astype(int)

    # ⭐ 순환형 인코딩 (시간 cos, sin)
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df['dayofweek_sin'] = np.sin(2 * np.pi * df['dayofweek'] / 7)
    df['dayofweek_cos'] = np.cos(2 * np.pi * df['dayofweek'] / 7)

    # 시간 기반 파생변수
    df['hour_squared'] = df['hour'] ** 2
    df['month_day_interaction'] = df['month'] * df['datetime'].dt.day
    
    # 기온 관련 파생변수
    if 'ta' in df.columns:
        df['ta_squared'] = df['ta'] ** 2
        df['ta_cubed'] = df['ta'] ** 3
    
    # 습도와 기온 상호작용
    if 'hm' in df.columns and 'ta' in df.columns:
        df['hm_ta_interaction'] = df['hm'] * df['ta']

    return df


# 파생변수 생성
train_df = create_features(train_df)
test_df = create_features(test_df)

print(f"✅ 파생변수 생성 완료: {train_df.shape[1]}개 컬럼")

✅ 파생변수 생성 완료: 37개 컬럼


## 3️⃣ 시즌별-브랜치별 데이터 분할

In [21]:
# 시즌별-브랜치별 데이터 분할
def split_by_season_and_branch(df):
    data_splits = {}
    
    branches = sorted(df['branch_id'].unique())
    seasons = [0, 1]  # 0: 비난방시즌, 1: 난방시즌
    season_names = {0: '비난방', 1: '난방'}
    
    print(f"📊 데이터 분할 정보:")
    print(f"   브랜치: {len(branches)}개 - {branches}")
    print(f"   시즌: {len(seasons)}개 - {[season_names[s] for s in seasons]}")
    print(f"   총 조합: {len(branches) * len(seasons)}개")
    
    for season in seasons:
        for branch in branches:
            key = f"{season_names[season]}_{branch}"
            
            # 시즌과 브랜치로 필터링
            subset = df[(df['heating_season'] == season) & (df['branch_id'] == branch)].copy()
            
            if len(subset) > 0:
                data_splits[key] = subset
                print(f"   {key}: {len(subset):,}개 데이터")
            else:
                print(f"   {key}: 데이터 없음 ⚠️")
    
    return data_splits

# 훈련 및 테스트 데이터 분할
train_splits = split_by_season_and_branch(train_df)
test_splits = split_by_season_and_branch(test_df)

print(f"\n✅ 훈련 데이터: {len(train_splits)}개 분할")
print(f"✅ 테스트 데이터: {len(test_splits)}개 분할")

📊 데이터 분할 정보:
   브랜치: 19개 - ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S']
   시즌: 2개 - ['비난방', '난방']
   총 조합: 38개
   비난방_A: 7,344개 데이터
   비난방_B: 7,344개 데이터
   비난방_C: 7,344개 데이터
   비난방_D: 7,344개 데이터
   비난방_E: 7,344개 데이터
   비난방_F: 7,344개 데이터
   비난방_G: 7,344개 데이터
   비난방_H: 7,344개 데이터
   비난방_I: 7,344개 데이터
   비난방_J: 7,344개 데이터
   비난방_K: 7,344개 데이터
   비난방_L: 7,344개 데이터
   비난방_M: 7,344개 데이터
   비난방_N: 7,344개 데이터
   비난방_O: 7,344개 데이터
   비난방_P: 7,344개 데이터
   비난방_Q: 7,344개 데이터
   비난방_R: 7,344개 데이터
   비난방_S: 7,344개 데이터
   난방_A: 10,175개 데이터
   난방_B: 10,175개 데이터
   난방_C: 10,175개 데이터
   난방_D: 10,175개 데이터
   난방_E: 10,175개 데이터
   난방_F: 10,175개 데이터
   난방_G: 10,175개 데이터
   난방_H: 10,175개 데이터
   난방_I: 10,175개 데이터
   난방_J: 10,175개 데이터
   난방_K: 10,175개 데이터
   난방_L: 10,175개 데이터
   난방_M: 10,175개 데이터
   난방_N: 10,175개 데이터
   난방_O: 10,175개 데이터
   난방_P: 10,175개 데이터
   난방_Q: 10,175개 데이터
   난방_R: 10,175개 데이터
   난방_S: 10,175개 데이터
📊 데이터 분할 정보:
   브랜치: 19개 - ['A', 'B', 'C

### 📌 Validation시 월별 추출

In [22]:
def get_temporal_split_indices(df, test_size=0.2, min_val_samples=10):
    """시간 기반 분할 인덱스 반환"""
    
    month_counts = df['month'].value_counts()
    valid_months = month_counts[month_counts >= min_val_samples * 2].index
    
    if len(valid_months) < 3:
        # 폴백: 기존 방식
        split_idx = int(len(df) * (1 - test_size))
        return df.index[:split_idx], df.index[split_idx:]
    
    train_indices = []
    val_indices = []
    
    for month in valid_months:
        month_data = df[df['month'] == month].sort_values('datetime')
        val_size = max(min_val_samples, int(len(month_data) * test_size))
        
        train_indices.extend(month_data.iloc[:-val_size].index)
        val_indices.extend(month_data.iloc[-val_size:].index)
    
    print(f"      📅 월별 분할: {len(valid_months)}개월, 검증 {len(val_indices)}개")
    
    return train_indices, val_indices

# *. 🔥 브랜치 선별 필터링 함수 (중간 추가)

In [23]:
# 1. 타겟 브랜치 정의 및 데이터 필터링 함수
TARGET_BRANCHES = ['A', 'B', 'C', 'D', 'G', 'H']
TARGET_SEASON = 'heating'  # 난방시즌만

def filter_target_splits(splits_dict, target_branches=TARGET_BRANCHES, target_season='난방'):
    """기존 train_splits/test_splits에서 타겟 브랜치만 필터링"""
    
    print(f"🎯 타겟 브랜치 필터링: {target_season}시즌 {target_branches}")
    
    filtered_splits = {}
    
    for key, data in splits_dict.items():
        # key 형태: "난방_A", "비난방_B" 등
        season, branch = key.split('_')
        
        if season == target_season and branch in target_branches:
            filtered_splits[key] = data
            print(f"   ✅ {key}: {len(data):,}개 데이터")
        else:
            print(f"   ⏭️ {key}: 건너뛰기")
    
    print(f"📊 필터링 결과: {len(filtered_splits)}개 모델 (기존 {len(splits_dict)}개)")
    return filtered_splits


# *. LSTM 데이터 전처리 함수

In [24]:
def create_lstm_sequences(X, y, sequence_length=24):
    """LSTM용 시퀀스 데이터 생성"""
    X_seq, y_seq = [], []
    
    for i in range(sequence_length, len(X)):
        X_seq.append(X.iloc[i-sequence_length:i].values)
        y_seq.append(y.iloc[i])
    
    return np.array(X_seq), np.array(y_seq)

def create_branch_lstm_data(df, feature_cols, target_col, sequence_length=24):
    """브랜치별 LSTM 시퀀스 데이터 생성"""
    # 시간순 정렬
    df_sorted = df.sort_values('datetime').copy()
    
    X = df_sorted[feature_cols]
    y = df_sorted[target_col]
    
    if len(X) > sequence_length:
        X_seq, y_seq = create_lstm_sequences(X, y, sequence_length)
        print(f"      📊 시퀀스 생성: {len(X_seq)}개 (원본 {len(X)}개)")
        return X_seq, y_seq
    else:
        print(f"      ⚠️ 데이터 부족: {len(X)}개 (최소 {sequence_length+1}개 필요)")
        return np.array([]), np.array([])

## 4️⃣ LSTM 모델 클래스 정의

In [25]:
class OptimizedLSTMModel:
    def __init__(self, model_name):
        self.model_name = model_name
        self.model = None
        self.best_params = None
        self.feature_cols = None
        self.study = None
        self.best_score = None
        self.sequence_length = 24
        self.scaler_X = MinMaxScaler()
        self.scaler_y = MinMaxScaler()
        
    def define_feature_columns(self, df):
        """특성 컬럼 정의"""
        exclude_cols = [
            'tm', 'datetime', 'year', 'heat_demand', 'branch_id'
        ]
        
        self.feature_cols = [col for col in df.columns 
                           if col not in exclude_cols and df[col].dtype in ['int64', 'float64']]
        
        print(f"   📋 {self.model_name}: {len(self.feature_cols)}개 특성 사용")
        return self.feature_cols
    
    def get_temporal_split_indices_lstm(self, df, test_size=0.2):
        """LSTM용 시간 기반 분할 (순차성 유지)"""
        df_sorted = df.sort_values('datetime')
        split_idx = int(len(df_sorted) * (1 - test_size))
        
        train_indices = df_sorted.index[:split_idx]
        val_indices = df_sorted.index[split_idx:]
        
        print(f"      📅 시계열 분할: 훈련 {len(train_indices)}개, 검증 {len(val_indices)}개")
        
        return train_indices, val_indices
    
    def build_lstm_model(self, input_shape, trial=None):
        """LSTM 모델 구조 정의"""
        if trial is not None:
            lstm_units_1 = trial.suggest_int('lstm_units_1', 32, 128)
            lstm_units_2 = trial.suggest_int('lstm_units_2', 16, 64)
            dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
            learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
            l2_reg = trial.suggest_float('l2_reg', 1e-6, 1e-3, log=True)
        else:
            lstm_units_1 = 64
            lstm_units_2 = 32
            dropout_rate = 0.3
            learning_rate = 0.001
            l2_reg = 1e-4
        
        model = Sequential([
            LSTM(lstm_units_1, 
                 return_sequences=True,
                 input_shape=input_shape,
                 kernel_regularizer=l2(l2_reg)),
            BatchNormalization(),
            Dropout(dropout_rate),
            
            LSTM(lstm_units_2,
                 return_sequences=False,
                 kernel_regularizer=l2(l2_reg)),
            BatchNormalization(),
            Dropout(dropout_rate),
            
            Dense(32, activation='relu', kernel_regularizer=l2(l2_reg)),
            Dropout(dropout_rate * 0.5),
            Dense(16, activation='relu'),
            Dense(1, activation='linear')
        ])
        
        optimizer = Adam(learning_rate=learning_rate)
        model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
        
        return model, {
            'lstm_units_1': lstm_units_1,
            'lstm_units_2': lstm_units_2,
            'dropout_rate': dropout_rate,
            'learning_rate': learning_rate,
            'l2_reg': l2_reg
        }
    
    def objective(self, trial, X_train_seq, y_train_seq, X_val_seq, y_val_seq):
        """Optuna 목적 함수"""
        
        input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])
        model, params = self.build_lstm_model(input_shape, trial)
        
        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
            verbose=0
        )
        
        model.fit(
            X_train_seq, y_train_seq,
            validation_data=(X_val_seq, y_val_seq),
            epochs=50,  # 시간 단축을 위해 축소
            batch_size=32,
            callbacks=[early_stopping],
            verbose=0
        )
        
        y_pred_scaled = model.predict(X_val_seq, verbose=0)
        y_pred = self.scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
        y_val_orig = self.scaler_y.inverse_transform(y_val_seq.reshape(-1, 1)).flatten()
        
        rmse = np.sqrt(mean_squared_error(y_val_orig, y_pred))
        
        del model
        tf.keras.backend.clear_session()
        
        return rmse
    
    def fit(self, df, target_col='heat_demand', n_trials=5):
        """모델 훈련 (기존 방식과 동일)"""
        print(f"\n🔍 {self.model_name} LSTM 모델 훈련 시작...")
        
        if len(df) < self.sequence_length * 3:
            print(f"   ⚠️ 데이터 부족 ({len(df)}개) - 기본 모델 사용")
            self._fit_basic_model(df, target_col)
            return
        
        # 1. 분할 인덱스 구하기
        train_indices, val_indices = self.get_temporal_split_indices_lstm(df, test_size=0.2)
        
        # 2. 특성 컬럼 정의
        self.define_feature_columns(df)
        
        # 3. 데이터 준비
        X = df[self.feature_cols].copy()
        y = df[target_col].copy()
        
        # 스케일러 피팅
        self.scaler_X.fit(X)
        self.scaler_y.fit(y.values.reshape(-1, 1))
        
        # 4. 훈련/검증 분할
        train_df = df.loc[train_indices]
        val_df = df.loc[val_indices]
        
        # 5. 시퀀스 데이터 생성
        X_train_seq, y_train_seq = create_branch_lstm_data(
            train_df, self.feature_cols, target_col, self.sequence_length
        )
        X_val_seq, y_val_seq = create_branch_lstm_data(
            val_df, self.feature_cols, target_col, self.sequence_length
        )
        
        if len(X_train_seq) == 0 or len(X_val_seq) == 0:
            print(f"   ⚠️ 시퀀스 생성 실패 - 기본 모델 사용")
            self._fit_basic_model(df, target_col)
            return
        
        # 6. 시퀀스 데이터 스케일링
        X_train_seq_scaled = np.zeros_like(X_train_seq)
        X_val_seq_scaled = np.zeros_like(X_val_seq)
        
        for i in range(X_train_seq.shape[0]):
            X_train_seq_scaled[i] = self.scaler_X.transform(X_train_seq[i])
        for i in range(X_val_seq.shape[0]):
            X_val_seq_scaled[i] = self.scaler_X.transform(X_val_seq[i])
            
        y_train_seq_scaled = self.scaler_y.transform(y_train_seq.reshape(-1, 1)).flatten()
        y_val_seq_scaled = self.scaler_y.transform(y_val_seq.reshape(-1, 1)).flatten()
        
        print(f"      📊 시퀀스 훈련: {len(X_train_seq_scaled):,}개, 검증: {len(X_val_seq_scaled):,}개")
        
        # Optuna 최적화
        print(f"   🎯 하이퍼파라미터 최적화: ", end="", flush=True)
        
        try:
            optuna.logging.set_verbosity(optuna.logging.ERROR)
            
            self.study = optuna.create_study(
                direction='minimize',
                sampler=TPESampler(seed=42),
                study_name=f"lstm_{self.model_name}"
            )
            
            def progress_callback(study, trial):
                print(f"\r   🎯 하이퍼파라미터 최적화: {len(study.trials)}/{n_trials} (Best RMSE: {study.best_value:.4f})", end="", flush=True)
            
            self.study.optimize(
                lambda trial: self.objective(trial, X_train_seq_scaled, y_train_seq_scaled, 
                                           X_val_seq_scaled, y_val_seq_scaled),
                n_trials=n_trials,
                callbacks=[progress_callback]
            )
            
            print()
            
            if len(self.study.trials) > 0 and self.study.best_trial is not None:
                self.best_score = self.study.best_value
                self.best_params = self.study.best_params.copy()
                
                # 최적 파라미터로 최종 모델 훈련
                input_shape = (X_train_seq_scaled.shape[1], X_train_seq_scaled.shape[2])
                self.model, _ = self.build_lstm_model(input_shape)
                
                # 전체 시퀀스 데이터로 훈련
                X_full_seq, y_full_seq = create_branch_lstm_data(
                    df, self.feature_cols, target_col, self.sequence_length
                )
                
                X_full_seq_scaled = np.zeros_like(X_full_seq)
                for i in range(X_full_seq.shape[0]):
                    X_full_seq_scaled[i] = self.scaler_X.transform(X_full_seq[i])
                y_full_seq_scaled = self.scaler_y.transform(y_full_seq.reshape(-1, 1)).flatten()
                
                early_stopping = EarlyStopping(
                    monitor='val_loss', patience=15, restore_best_weights=True, verbose=0
                )
                
                self.model.fit(
                    X_full_seq_scaled, y_full_seq_scaled,
                    validation_data=(X_val_seq_scaled, y_val_seq_scaled),
                    epochs=100,
                    batch_size=32,
                    callbacks=[early_stopping],
                    verbose=0
                )
                
                # 성능 정보
                val_pred_scaled = self.model.predict(X_val_seq_scaled, verbose=0)
                val_pred = self.scaler_y.inverse_transform(val_pred_scaled.reshape(-1, 1)).flatten()
                y_val_orig = self.scaler_y.inverse_transform(y_val_seq_scaled.reshape(-1, 1)).flatten()
                val_rmse = np.sqrt(mean_squared_error(y_val_orig, val_pred))
                
                print(f"   📈 최적화 완료: Best RMSE = {self.best_score:.4f}")
                print(f"   📊 검증 RMSE = {val_rmse:.4f}")
                print(f"   🏆 최적 파라미터: lr={self.best_params.get('learning_rate', 0.001):.5f}, units={self.best_params.get('lstm_units_1', 64)}")
                
            else:
                print(f"   ⚠️ 최적화 실패: 유효한 trial 없음 - 기본 모델 사용")
                self._fit_basic_model(df, target_col)
                
        except Exception as e:
            print(f"\n   ⚠️ 최적화 실패: {str(e)[:30]}... - 기본 모델 사용")
            self._fit_basic_model(df, target_col)
    
    def _fit_basic_model(self, df, target_col):
        """기본 모델 훈련"""
        try:
            X_full_seq, y_full_seq = create_branch_lstm_data(
                df, self.feature_cols, target_col, self.sequence_length
            )
            
            if len(X_full_seq) == 0:
                print(f"   ❌ 시퀀스 데이터 생성 실패")
                self.model = None
                return
            
            X_full_seq_scaled = np.zeros_like(X_full_seq)
            for i in range(X_full_seq.shape[0]):
                X_full_seq_scaled[i] = self.scaler_X.transform(X_full_seq[i])
            y_full_seq_scaled = self.scaler_y.transform(y_full_seq.reshape(-1, 1)).flatten()
            
            input_shape = (X_full_seq_scaled.shape[1], X_full_seq_scaled.shape[2])
            self.model, _ = self.build_lstm_model(input_shape)
            
            self.model.fit(X_full_seq_scaled, y_full_seq_scaled, epochs=30, batch_size=32, verbose=0)
            
            self.best_score = None
            self.best_params = None
            print(f"   🔧 기본 LSTM 모델 훈련 완료")
            
        except Exception as e:
            print(f"   ❌ 기본 모델 훈련 실패: {str(e)[:30]}...")
            self.model = None
    
    def predict(self, df):
        """예측"""
        if self.model is None:
            return np.full(len(df), 0)
        
        try:
            X_seq, _ = create_branch_lstm_data(
                df, self.feature_cols, 'heat_demand', self.sequence_length
            )
            
            if len(X_seq) == 0:
                return np.full(len(df), 0)
            
            X_seq_scaled = np.zeros_like(X_seq)
            for i in range(X_seq.shape[0]):
                X_seq_scaled[i] = self.scaler_X.transform(X_seq[i])
            
            pred_scaled = self.model.predict(X_seq_scaled, verbose=0)
            predictions = self.scaler_y.inverse_transform(pred_scaled.reshape(-1, 1)).flatten()
            
            # 시퀀스 길이만큼 앞의 데이터는 0으로 패딩
            full_predictions = np.zeros(len(df))
            full_predictions[self.sequence_length:self.sequence_length+len(predictions)] = predictions
            
            return np.maximum(full_predictions, 0)
            
        except Exception as e:
            print(f"   ❌ 예측 실패: {str(e)[:30]}...")
            return np.full(len(df), 0)

print("✅ LSTM 모델 클래스 정의 완료")

✅ LSTM 모델 클래스 정의 완료


## 5️⃣ 일단 일부만 모델 훈련

In [26]:
# 🔥 6개 브랜치 LSTM 모델 훈련
print("🚀 6개 브랜치 LSTM 모델 훈련 시작!")
print("=" * 60)

# 1. 타겟 브랜치 필터링
TARGET_BRANCHES = ['A', 'B', 'C', 'D', 'G', 'H']
filtered_train_splits = filter_target_splits(train_splits, TARGET_BRANCHES, '난방')
filtered_test_splits = filter_target_splits(test_splits, TARGET_BRANCHES, '난방')

# 2. 6개 모델 훈련 (기존과 동일한 구조)
models = {}
training_results = {}
n_trials_per_model = 5  # LSTM은 시간이 오래 걸리므로 축소

start_time = datetime.now()
success_count = 0
failed_count = 0

for i, (model_key, train_data) in enumerate(filtered_train_splits.items(), 1):
    print(f"\n[{i:2d}/{len(filtered_train_splits)}] 🔥 {model_key}")
    print(f"         📊 데이터: {len(train_data):,}개")
    
    # LSTM 모델 생성 및 훈련
    model = OptimizedLSTMModel(model_key)
    
    try:
        model_start = datetime.now()
        model.fit(train_data, n_trials=n_trials_per_model)
        model_time = (datetime.now() - model_start).total_seconds()
        
        models[model_key] = model
        
        if model.best_score is not None:
            success_count += 1
            status = "✅ 최적화"
            score_text = f"RMSE: {model.best_score:.3f}"
        else:
            status = "🔧 기본모델"
            score_text = "기본파라미터"
        
        training_results[model_key] = {
            'data_size': len(train_data),
            'training_time': model_time,
            'best_score': model.best_score,
            'best_params': model.best_params,
            'optimization_success': model.best_score is not None
        }
        
        print(f"         {status} | {score_text} | ⏱️ {model_time:.1f}초")
        
    except Exception as e:
        print(f"         ❌ 훈련 실패: {str(e)[:30]}...")
        failed_count += 1
        
        # 더미 모델 저장
        dummy_model = OptimizedLSTMModel(model_key)
        dummy_model.model = None
        models[model_key] = dummy_model
        
        training_results[model_key] = {
            'data_size': len(train_data),
            'training_time': 0,
            'best_score': None,
            'best_params': None,
            'optimization_success': False,
            'error': str(e)[:50]
        }

total_time = (datetime.now() - start_time).total_seconds()

print(f"\n" + "=" * 60)
print(f"🎉 6개 LSTM 모델 훈련 완료!")
print(f"⏱️  총 소요 시간: {total_time/60:.1f}분")
print(f"📊 훈련 결과:")
print(f"   ✅ 성공: {success_count}개")
print(f"   ❌ 실패: {failed_count}개")
print(f"   🎯 대상: 난방시즌 {TARGET_BRANCHES} 브랜치")

🚀 6개 브랜치 LSTM 모델 훈련 시작!
🎯 타겟 브랜치 필터링: 난방시즌 ['A', 'B', 'C', 'D', 'G', 'H']
   ⏭️ 비난방_A: 건너뛰기
   ⏭️ 비난방_B: 건너뛰기
   ⏭️ 비난방_C: 건너뛰기
   ⏭️ 비난방_D: 건너뛰기
   ⏭️ 비난방_E: 건너뛰기
   ⏭️ 비난방_F: 건너뛰기
   ⏭️ 비난방_G: 건너뛰기
   ⏭️ 비난방_H: 건너뛰기
   ⏭️ 비난방_I: 건너뛰기
   ⏭️ 비난방_J: 건너뛰기
   ⏭️ 비난방_K: 건너뛰기
   ⏭️ 비난방_L: 건너뛰기
   ⏭️ 비난방_M: 건너뛰기
   ⏭️ 비난방_N: 건너뛰기
   ⏭️ 비난방_O: 건너뛰기
   ⏭️ 비난방_P: 건너뛰기
   ⏭️ 비난방_Q: 건너뛰기
   ⏭️ 비난방_R: 건너뛰기
   ⏭️ 비난방_S: 건너뛰기
   ✅ 난방_A: 10,175개 데이터
   ✅ 난방_B: 10,175개 데이터
   ✅ 난방_C: 10,175개 데이터
   ✅ 난방_D: 10,175개 데이터
   ⏭️ 난방_E: 건너뛰기
   ⏭️ 난방_F: 건너뛰기
   ✅ 난방_G: 10,175개 데이터
   ✅ 난방_H: 10,175개 데이터
   ⏭️ 난방_I: 건너뛰기
   ⏭️ 난방_J: 건너뛰기
   ⏭️ 난방_K: 건너뛰기
   ⏭️ 난방_L: 건너뛰기
   ⏭️ 난방_M: 건너뛰기
   ⏭️ 난방_N: 건너뛰기
   ⏭️ 난방_O: 건너뛰기
   ⏭️ 난방_P: 건너뛰기
   ⏭️ 난방_Q: 건너뛰기
   ⏭️ 난방_R: 건너뛰기
   ⏭️ 난방_S: 건너뛰기
📊 필터링 결과: 6개 모델 (기존 38개)
🎯 타겟 브랜치 필터링: 난방시즌 ['A', 'B', 'C', 'D', 'G', 'H']
   ⏭️ 비난방_A: 건너뛰기
   ⏭️ 비난방_B: 건너뛰기
   ⏭️ 비난방_C: 건너뛰기
   ⏭️ 비난방_D: 건너뛰기
   ⏭️ 비난방_E: 건너뛰기
   ⏭️ 비난방_F: 건너뛰기
   ⏭️ 비난방_G: 건너뛰기
   ⏭️ 비난방_H: 건너뛰기
   ⏭️ 

## 6️⃣ 훈련 결과 분석

In [27]:
def analyze_lstm_training_results(training_results, models):
    """6개 LSTM 모델 훈련 결과 분석"""
    print("\n📊 LSTM 훈련 결과 분석")
    print("=" * 60)
    
    # 결과 정리
    results_list = []
    for model_name, result in training_results.items():
        # best_score를 안전하게 변환
        best_score = result.get('best_score')
        if best_score is not None:
            try:
                best_score = float(best_score)
            except (ValueError, TypeError):
                best_score = None
        
        # 안전한 딕셔너리 생성
        safe_result = {
            'model_name': model_name,
            'branch': model_name.split('_')[1] if '_' in model_name else 'unknown',
            'data_size': result.get('data_size', 0),
            'training_time': result.get('training_time', 0),
            'best_score': best_score,
            'optimization_success': result.get('optimization_success', False),
            'has_model': models.get(model_name) is not None and models.get(model_name).model is not None
        }
        results_list.append(safe_result)
    
    # DataFrame 생성
    results_df = pd.DataFrame(results_list)
    
    # 성공/실패 통계
    successful_models = results_df[results_df['optimization_success'] == True]
    basic_models = results_df[(results_df['optimization_success'] == False) & (results_df['has_model'] == True)]
    failed_models = results_df[results_df['has_model'] == False]
    
    print(f"✅ 최적화 성공: {len(successful_models)}개 모델")
    print(f"🔧 기본 모델: {len(basic_models)}개 모델")
    print(f"❌ 완전 실패: {len(failed_models)}개 모델")
    
    # 성공한 모델들의 성능 통계
    if len(successful_models) > 0:
        valid_scores = successful_models[successful_models['best_score'].notna()]
        
        if len(valid_scores) > 0:
            print(f"\n🏆 LSTM 최적화 성능 통계:")
            print(f"   평균 RMSE: {valid_scores['best_score'].mean():.4f}")
            print(f"   최소 RMSE: {valid_scores['best_score'].min():.4f}")
            print(f"   최대 RMSE: {valid_scores['best_score'].max():.4f}")
            print(f"   표준편차: {valid_scores['best_score'].std():.4f}")
            
            # 브랜치별 성능
            print(f"\n📈 브랜치별 RMSE:")
            for _, row in valid_scores.iterrows():
                print(f"   브랜치 {row['branch']}: RMSE = {row['best_score']:.4f} | 데이터 = {row['data_size']:,}개")
            
            # 성능 순위
            print(f"\n🥇 성능 순위 (RMSE 낮은 순):")
            try:
                # best_score를 숫자형으로 확실히 변환
                valid_scores_copy = valid_scores.copy()
                valid_scores_copy['best_score'] = pd.to_numeric(valid_scores_copy['best_score'], errors='coerce')
                
                # NaN 제거 후 정렬
                top_models = valid_scores_copy.dropna(subset=['best_score']).nsmallest(6, 'best_score')
                
                for idx, (_, row) in enumerate(top_models.iterrows(), 1):
                    print(f"   {idx}. 브랜치 {row['branch']}: RMSE = {row['best_score']:.4f}")
                    
            except Exception as e:
                print(f"   ⚠️ 순위 정렬 실패: {str(e)}")
                # 대안: 수동 정렬
                try:
                    scores_list = [(row['branch'], row['best_score']) 
                                 for _, row in valid_scores.iterrows() 
                                 if pd.notna(row['best_score'])]
                    scores_list.sort(key=lambda x: float(x[1]))
                    
                    for idx, (branch, score) in enumerate(scores_list, 1):
                        print(f"   {idx}. 브랜치 {branch}: RMSE = {score:.4f}")
                except Exception as e2:
                    print(f"   ❌ 수동 정렬도 실패: {str(e2)}")
    
    # 실패한 모델 정보
    if len(failed_models) > 0:
        print(f"\n⚠️ 실패한 모델들:")
        for _, row in failed_models.iterrows():
            error_msg = training_results[row['model_name']].get('error', '알 수 없는 오류')
            print(f"   브랜치 {row['branch']}: {error_msg}")
    
    # 훈련 시간 통계
    avg_time = results_df['training_time'].mean()
    total_time_min = results_df['training_time'].sum() / 60
    print(f"\n⏱️ 훈련 시간 통계:")
    print(f"   평균 모델당: {avg_time:.1f}초")
    print(f"   총 훈련 시간: {total_time_min:.1f}분")
    
    # 데이터 크기별 성능 분석
    if len(successful_models) > 0:
        print(f"\n📊 데이터 크기 vs 성능 분석:")
        for _, row in successful_models.iterrows():
            if pd.notna(row['best_score']):
                data_per_1k = row['data_size'] / 1000
                print(f"   브랜치 {row['branch']}: {data_per_1k:.1f}k개 → RMSE {row['best_score']:.4f}")
    
    # LSTM 모델 특성 분석
    print(f"\n🔬 LSTM 모델 특성:")
    print(f"   시퀀스 길이: 24시간")
    print(f"   학습 대상: 난방시즌만")
    print(f"   타겟 브랜치: A, B, C, D, G, H")
    
    # 하이퍼파라미터 분석 (성공한 모델들)
    if len(successful_models) > 0:
        print(f"\n🎯 최적화된 하이퍼파라미터 분석:")
        for _, row in successful_models.iterrows():
            model = models.get(row['model_name'])
            if model and model.best_params:
                params = model.best_params
                lr = params.get('learning_rate', 'N/A')
                units1 = params.get('lstm_units_1', 'N/A')
                units2 = params.get('lstm_units_2', 'N/A')
                dropout = params.get('dropout_rate', 'N/A')
                print(f"   브랜치 {row['branch']}: lr={lr:.5f}, units=({units1},{units2}), dropout={dropout:.3f}")
    
    return results_df, successful_models, failed_models

# 실행 함수
def run_lstm_analysis():
    """LSTM 훈련 결과 분석 실행"""
    results_df, successful_models, failed_models = analyze_lstm_training_results(training_results, models)
    return results_df, successful_models, failed_models

print("✅ LSTM 훈련 결과 분석 함수 준비 완료")

# LSTM 훈련 결과 분석 실행
results_df, successful_models, failed_models = run_lstm_analysis()

✅ LSTM 훈련 결과 분석 함수 준비 완료

📊 LSTM 훈련 결과 분석
✅ 최적화 성공: 6개 모델
🔧 기본 모델: 0개 모델
❌ 완전 실패: 0개 모델

🏆 LSTM 최적화 성능 통계:
   평균 RMSE: 33.3423
   최소 RMSE: 20.9227
   최대 RMSE: 46.5207
   표준편차: 8.5127

📈 브랜치별 RMSE:
   브랜치 A: RMSE = 20.9227 | 데이터 = 10,175개
   브랜치 B: RMSE = 46.5207 | 데이터 = 10,175개
   브랜치 C: RMSE = 33.7062 | 데이터 = 10,175개
   브랜치 D: RMSE = 32.4766 | 데이터 = 10,175개
   브랜치 G: RMSE = 29.1212 | 데이터 = 10,175개
   브랜치 H: RMSE = 37.3062 | 데이터 = 10,175개

🥇 성능 순위 (RMSE 낮은 순):
   1. 브랜치 A: RMSE = 20.9227
   2. 브랜치 G: RMSE = 29.1212
   3. 브랜치 D: RMSE = 32.4766
   4. 브랜치 C: RMSE = 33.7062
   5. 브랜치 H: RMSE = 37.3062
   6. 브랜치 B: RMSE = 46.5207

⏱️ 훈련 시간 통계:
   평균 모델당: 920.2초
   총 훈련 시간: 92.0분

📊 데이터 크기 vs 성능 분석:
   브랜치 A: 10.2k개 → RMSE 20.9227
   브랜치 B: 10.2k개 → RMSE 46.5207
   브랜치 C: 10.2k개 → RMSE 33.7062
   브랜치 D: 10.2k개 → RMSE 32.4766
   브랜치 G: 10.2k개 → RMSE 29.1212
   브랜치 H: 10.2k개 → RMSE 37.3062

🔬 LSTM 모델 특성:
   시퀀스 길이: 24시간
   학습 대상: 난방시즌만
   타겟 브랜치: A, B, C, D, G, H

🎯 최적화된 하이퍼파라미터 분석:
   브랜치 A: 

## 7️⃣ 테스트 예측

In [28]:
# 시즌별 예측 (filtered_test_splits 사용)
print("🎯 6개 LSTM 모델 예측 시작...")

predictions = {}

for model_key, model in models.items():
    if model_key in filtered_test_splits:
        test_data = filtered_test_splits[model_key]
        
        print(f"📊 {model_key}: {len(test_data):,}개 데이터 예측 중...")
        
        try:
            pred = model.predict(test_data)
            predictions[model_key] = {
                'data': test_data,
                'predictions': pred
            }
            print(f"   ✅ 완료: 평균={np.mean(pred):.2f}")
            
        except Exception as e:
            print(f"   ❌ 예측 실패: {str(e)[:30]}...")
            predictions[model_key] = {
                'data': test_data,
                'predictions': np.zeros(len(test_data))
            }

print(f"✅ 예측 완료: {len(predictions)}개 모델")


🎯 6개 LSTM 모델 예측 시작...
📊 난방_A: 5,088개 데이터 예측 중...
      📊 시퀀스 생성: 5064개 (원본 5088개)
   ✅ 완료: 평균=162.31
📊 난방_B: 5,088개 데이터 예측 중...
      📊 시퀀스 생성: 5064개 (원본 5088개)
   ✅ 완료: 평균=348.25
📊 난방_C: 5,088개 데이터 예측 중...
      📊 시퀀스 생성: 5064개 (원본 5088개)
   ✅ 완료: 평균=355.05
📊 난방_D: 5,088개 데이터 예측 중...
      📊 시퀀스 생성: 5064개 (원본 5088개)
   ✅ 완료: 평균=233.65
📊 난방_G: 5,088개 데이터 예측 중...
      📊 시퀀스 생성: 5064개 (원본 5088개)
   ✅ 완료: 평균=301.11
📊 난방_H: 5,088개 데이터 예측 중...
      📊 시퀀스 생성: 5064개 (원본 5088개)
   ✅ 완료: 평균=207.50
✅ 예측 완료: 6개 모델


## 8️⃣ 예측 결과 통합

In [29]:
# 예측 결과를 원본 test_df 순서에 맞게 통합
print("🔄 예측 결과 통합 중...")

# 결과를 저장할 배열 초기화
final_predictions = np.zeros(len(test_df))
prediction_counts = np.zeros(len(test_df))  # 각 인덱스별 예측 횟수 추적

# 각 예측 결과를 해당 인덱스에 할당
for model_key, pred_info in predictions.items():
    test_data = pred_info['data']
    pred_values = pred_info['predictions']
    
    # 원본 test_df에서 해당 데이터의 인덱스 찾기
    season, branch = model_key.split('_')
    season_num = 1 if season == '난방' else 0
    
    # 조건에 맞는 인덱스 찾기
    mask = (test_df['heating_season'] == season_num) & (test_df['branch_id'] == branch)
    indices = test_df[mask].index.tolist()
    
    print(f"📊 {model_key}: {len(indices)}개 인덱스에 할당")
    
    # 예측값 할당 (인덱스 개수와 예측값 개수가 맞는지 확인)
    if len(indices) == len(pred_values):
        for i, idx in enumerate(indices):
            final_predictions[idx] = pred_values[i]
            prediction_counts[idx] += 1
    else:
        print(f"   ⚠️ 크기 불일치: 인덱스 {len(indices)}개 vs 예측값 {len(pred_values)}개")
        # 크기가 다르면 최소 개수만큼만 할당
        min_len = min(len(indices), len(pred_values))
        for i in range(min_len):
            final_predictions[indices[i]] = pred_values[i]
            prediction_counts[indices[i]] += 1

# 예측되지 않은 데이터 확인
unassigned_count = np.sum(prediction_counts == 0)
if unassigned_count > 0:
    print(f"⚠️ 예측되지 않은 데이터: {unassigned_count}개 (0으로 유지)")

# 중복 예측 확인
duplicate_count = np.sum(prediction_counts > 1)
if duplicate_count > 0:
    print(f"⚠️ 중복 예측된 데이터: {duplicate_count}개")

print(f"\n✅ 예측 결과 통합 완료")
print(f"   📊 총 예측 개수: {len(final_predictions):,}개")
print(f"   📈 예측값 통계: 평균={np.mean(final_predictions):.2f}, 최대={np.max(final_predictions):.2f}")

🔄 예측 결과 통합 중...
📊 난방_A: 5088개 인덱스에 할당
📊 난방_B: 5088개 인덱스에 할당
📊 난방_C: 5088개 인덱스에 할당
📊 난방_D: 5088개 인덱스에 할당
📊 난방_G: 5088개 인덱스에 할당
📊 난방_H: 5088개 인덱스에 할당
⚠️ 예측되지 않은 데이터: 135912개 (0으로 유지)

✅ 예측 결과 통합 완료
   📊 총 예측 개수: 166,440개
   📈 예측값 통계: 평균=49.15, 최대=892.82


## 9️⃣ 최종 결과 저장 및 평가

In [30]:
# 최종 결과를 test_df에 추가
print("💾 최종 결과 저장...")

# 원본 test_df 복사
result_df = test_df.copy()

# 예측 결과 추가 (음수값 제거)
result_df['pred_heat_demand'] = np.maximum(final_predictions, 0).round(1)

# CSV 파일 저장
output_filename = 'lightgbm_branch_season_predictions.csv'
result_df.to_csv(output_filename, index=False)

print(f"📁 결과 파일 저장: {output_filename}")

# =============================================================================
# RMSE 중심 성능 평가 (실제값이 있는 경우)
# =============================================================================

if 'heat_demand' in test_df.columns:
    print(f"\n📊 RMSE 성능 평가")
    print("=" * 60)
    
    # 전체 RMSE
    y_true = test_df['heat_demand'].values
    y_pred = result_df['pred_heat_demand'].values
    
    # 음수나 NaN 값 제거
    valid_mask = ~(np.isnan(y_true) | np.isnan(y_pred))
    y_true_clean = y_true[valid_mask]
    y_pred_clean = y_pred[valid_mask]
    
    overall_rmse = np.sqrt(mean_squared_error(y_true_clean, y_pred_clean))
    overall_mae = mean_absolute_error(y_true_clean, y_pred_clean)
    correlation = np.corrcoef(y_true_clean, y_pred_clean)[0, 1]
    
    print(f"🏆 전체 성능:")
    print(f"   RMSE: {overall_rmse:.4f}")
    print(f"   MAE:  {overall_mae:.4f}")
    print(f"   상관계수: {correlation:.4f}")
    print(f"   유효 데이터: {len(y_true_clean):,}개")
    
    # 시즌별 RMSE (핵심!)
    print(f"\n📈 시즌별 RMSE 성능:")
    season_names = {0: '비난방시즌', 1: '난방시즌'}
    season_results = {}
    
    for season in [0, 1]:
        mask = (test_df['heating_season'] == season) & valid_mask
        if np.sum(mask) > 0:
            season_rmse = np.sqrt(mean_squared_error(y_true[mask], y_pred[mask]))
            season_mae = mean_absolute_error(y_true[mask], y_pred[mask])
            season_corr = np.corrcoef(y_true[mask], y_pred[mask])[0, 1] if np.sum(mask) > 1 else 0
            season_results[season] = {
                'rmse': season_rmse, 
                'mae': season_mae, 
                'corr': season_corr,
                'count': np.sum(mask)
            }
            
            print(f"   {season_names[season]:8s}: RMSE={season_rmse:7.4f} | MAE={season_mae:7.4f} | 상관={season_corr:6.3f} | {np.sum(mask):,}개")
    
    # 브랜치별 RMSE (상위/하위 분석)
    print(f"\n📊 브랜치별 RMSE 성능:")
    branch_results = {}
    
    for branch in sorted(test_df['branch_id'].unique()):
        mask = (test_df['branch_id'] == branch) & valid_mask
        if np.sum(mask) > 1:  # 최소 2개 이상의 데이터가 있어야 RMSE 계산 가능
            branch_rmse = np.sqrt(mean_squared_error(y_true[mask], y_pred[mask]))
            branch_mae = mean_absolute_error(y_true[mask], y_pred[mask])
            branch_results[branch] = {
                'rmse': branch_rmse,
                'mae': branch_mae, 
                'count': np.sum(mask)
            }
    
    if branch_results:
        # RMSE 기준 정렬
        sorted_branches = sorted(branch_results.items(), key=lambda x: x[1]['rmse'])
        
        print(f"   🥇 RMSE 우수 브랜치 (Top 5):")
        for i, (branch, metrics) in enumerate(sorted_branches[:5], 1):
            print(f"      {i}. 브랜치 {branch}: RMSE={metrics['rmse']:7.4f} | MAE={metrics['mae']:7.4f} | {metrics['count']:,}개")
        
        print(f"   🥉 RMSE 개선 필요 브랜치 (Bottom 5):")
        for i, (branch, metrics) in enumerate(sorted_branches[-5:], 1):
            print(f"      {i}. 브랜치 {branch}: RMSE={metrics['rmse']:7.4f} | MAE={metrics['mae']:7.4f} | {metrics['count']:,}개")
        
        # 브랜치별 성능 통계
        rmse_values = [v['rmse'] for v in branch_results.values()]
        print(f"\n   📈 브랜치별 RMSE 통계:")
        print(f"      평균: {np.mean(rmse_values):.4f}")
        print(f"      표준편차: {np.std(rmse_values):.4f}")
        print(f"      최소: {np.min(rmse_values):.4f}")
        print(f"      최대: {np.max(rmse_values):.4f}")
    
    # 시즌×브랜치 조합별 RMSE (상위 10개만)
    print(f"\n🔥 시즌×브랜치 조합별 RMSE (Ranking):")
    combo_results = []
    
    for season in [0, 1]:
        for branch in test_df['branch_id'].unique():
            mask = (test_df['heating_season'] == season) & (test_df['branch_id'] == branch) & valid_mask
            if np.sum(mask) > 1:
                combo_rmse = np.sqrt(mean_squared_error(y_true[mask], y_pred[mask]))
                combo_name = f"{season_names[season]}_{branch}"
                combo_results.append((combo_name, combo_rmse, np.sum(mask)))
    
    # RMSE 기준 정렬하여 표시
    combo_results.sort(key=lambda x: x[1])
    for i, (combo_name, rmse, count) in enumerate(combo_results, 1):
        print(f"   {i:2d}. {combo_name:15s}: RMSE={rmse:7.4f} | {count:,}개")
    
    # 훈련된 모델들의 최적화 성능과 실제 테스트 성능 비교
    print(f"\n🔍 모델 최적화 vs 실제 성능 비교:")
    optimization_rmses = [v['best_score'] for v in training_results.values() if v.get('best_score') is not None]
    
    if optimization_rmses:
        print(f"   훈련시 최적화 RMSE: 평균={np.mean(optimization_rmses):.4f}, 범위=[{np.min(optimization_rmses):.4f}, {np.max(optimization_rmses):.4f}]")
        print(f"   실제 테스트 RMSE: {overall_rmse:.4f}")
        print(f"   성능 차이: {abs(overall_rmse - np.mean(optimization_rmses)):.4f}")
    
else:
    print(f"\n⚠️ 테스트 데이터에 heat_demand 컬럼이 없어서 RMSE 평가를 수행할 수 없습니다.")
    print(f"   예측 결과만 저장되었습니다.")
    
    # 예측값 기본 통계
    print(f"\n📊 예측값 기본 통계:")
    print(f"   개수: {len(result_df):,}개")
    print(f"   평균: {result_df['pred_heat_demand'].mean():.2f}")
    print(f"   중앙값: {result_df['pred_heat_demand'].median():.2f}")
    print(f"   표준편차: {result_df['pred_heat_demand'].std():.2f}")
    print(f"   범위: [{result_df['pred_heat_demand'].min():.2f}, {result_df['pred_heat_demand'].max():.2f}]")
    
    # 시즌별 예측 통계
    print(f"\n📈 시즌별 예측 통계:")
    for season in [0, 1]:
        season_data = result_df[result_df['heating_season'] == season]['pred_heat_demand']
        if len(season_data) > 0:
            season_name = '비난방시즌' if season == 0 else '난방시즌'
            print(f"   {season_name}: 평균={season_data.mean():.2f}, 개수={len(season_data):,}개")

print("=" * 60)

💾 최종 결과 저장...
📁 결과 파일 저장: lightgbm_branch_season_predictions.csv

📊 RMSE 성능 평가
🏆 전체 성능:
   RMSE: 70.8864
   MAE:  52.6478
   상관계수: 0.8915
   유효 데이터: 166,440개

📈 시즌별 RMSE 성능:
   비난방시즌   : RMSE=51.8487 | MAE=40.9144 | 상관=   nan | 69,768개
   난방시즌    : RMSE=81.9218 | MAE=61.1158 | 상관= 0.896 | 96,672개

📊 브랜치별 RMSE 성능:
   🥇 RMSE 우수 브랜치 (Top 5):
      1. 브랜치 R: RMSE=17.1769 | MAE=14.6325 | 8,760개
      2. 브랜치 S: RMSE=17.5521 | MAE=15.1737 | 8,760개
      3. 브랜치 L: RMSE=29.6589 | MAE=23.1355 | 8,760개
      4. 브랜치 A: RMSE=31.2909 | MAE=25.5237 | 8,760개
      5. 브랜치 M: RMSE=44.5678 | MAE=35.3445 | 8,760개
   🥉 RMSE 개선 필요 브랜치 (Bottom 5):
      1. 브랜치 O: RMSE=85.3509 | MAE=65.9368 | 8,760개
      2. 브랜치 I: RMSE=91.9005 | MAE=74.7925 | 8,760개
      3. 브랜치 J: RMSE=102.2785 | MAE=87.2439 | 8,760개
      4. 브랜치 N: RMSE=105.3003 | MAE=79.9602 | 8,760개
      5. 브랜치 P: RMSE=116.2071 | MAE=96.9252 | 8,760개

   📈 브랜치별 RMSE 통계:
      평균: 65.0004
      표준편차: 28.2812
      최소: 17.1769
      최대: 116.2071

🔥 시즌×브랜치

## 🔟 모델 정보 저장

In [31]:
# 모델 정보 및 결과 저장
print("💾 모델 정보 저장...")

# 훈련 결과 및 모델 정보를 JSON으로 저장
model_info = {
    'total_models': len(models),
    'successful_models': len([k for k, v in training_results.items() if v.get('best_score') is not None]),
    'training_results': training_results,
    'prediction_stats': prediction_stats if 'prediction_stats' in locals() else {},
    'feature_columns': models[list(models.keys())[0]].feature_cols if models else [],
    'optimization_trials_per_model': n_trials_per_model,
    'total_training_time_minutes': total_time_min if 'total_time_min' in locals() else 0
}

# RMSE 결과 추가 (있는 경우)
if 'heat_demand' in test_df.columns:
    model_info['evaluation_results'] = {
        'overall_rmse': overall_rmse,
        'overall_mae': overall_mae,
        'correlation': correlation
    }

# JSON 파일로 저장
with open('model_info_and_results.json', 'w', encoding='utf-8') as f:
    json.dump(model_info, f, indent=2, ensure_ascii=False, default=str)

print("📁 model_info_and_results.json 저장 완료")

# 간단한 요약 출력
print(f"\n📋 모델 정보 요약:")
print(f"   🔧 총 모델 수: {model_info['total_models']}개")
print(f"   ✅ 성공한 모델: {model_info['successful_models']}개")
print(f"   📊 사용 특성 수: {len(model_info['feature_columns'])}개")
print(f"   🎯 모델당 최적화 시도: {model_info['optimization_trials_per_model']}회")

# Google Drive 저장 (Colab 환경)
if IN_COLAB:
    save_drive = input("\nGoogle Drive에 결과 파일들을 저장하시겠습니까? (y/n): ").lower() == 'y'
    if save_drive:
        try:
            !cp {output_filename} /content/drive/MyDrive/
            !cp model_info_and_results.json /content/drive/MyDrive/
            print("✅ Google Drive 저장 완료!")
        except Exception as e:
            print(f"⚠️ Google Drive 저장 실패: {e}")

💾 모델 정보 저장...


NameError: name 'json' is not defined

## 🎯 최종 요약

In [ ]:
print("\n" + "="*80)
print("🔥 지역난방 열수요 예측: 시즌별-브랜치별 XGBoost 모델 - 최종 요약")
print("="*80)

print(f"\n🏗️ 모델 구성:")
print(f"   📊 XGBoost 개별 모델 (시즌별 × 브랜치별)")
print(f"   🎯 Optuna TPE 하이퍼파라미터 최적화")
print(f"   📋 총 모델 수: {len(models)}개")
print(f"   ✅ 성공적 훈련: {len([k for k, v in training_results.items() if v.get('best_score') is not None])}개")

print(f"\n📈 특성 엔지니어링:")
print(f"   ⭐ HDD, wind_chill, 온도 제곱/세제곱")
print(f"   🔄 순환형 인코딩 (시간, 월, 요일)")
print(f"   📋 범주형: heating_season, 피크시간, 기온범주, 강수강도")
print(f"   🧮 상호작용: 습도×기온, 월×일")
# print(f"   📏 StandardScaler 정규화")

print(f"\n🎯 모델링 전략:")
print(f"   ❄️ 난방시즌 (10,11,12,1,2,3,4월) 전용 모델")
print(f"   🌞 비난방시즌 (5,6,7,8,9월) 전용 모델")
print(f"   🏢 브랜치별 개별 모델 (각 지사의 특성 반영)")
print(f"   ⚡ XGBoost with Early Stopping")

print(f"\n🔍 최적화 설정:")
print(f"   📊 Optuna TPE Sampler")
print(f"   🎯 모델당 {n_trials_per_model}회 시도")
print(f"   📈 시계열 기반 Train/Validation 분할 (80:20)")
print(f"   🎪 XGBoost Pruning Callback 사용")

if 'heat_demand' in test_df.columns:
    print(f"\n🏆 최종 성능:")
    print(f"   📊 전체 RMSE: {overall_rmse:.4f}")
    print(f"   📏 전체 MAE: {overall_mae:.4f}")
    print(f"   📈 상관계수: {correlation:.4f}")

# 최고 성능 모델 정보
if successful_models is not None and len(successful_models) > 0:
    best_model_name = successful_models['best_score'].idxmin()
    best_score = successful_models.loc[best_model_name, 'best_score']
    print(f"\n🥇 최고 성능 모델: {best_model_name} (RMSE: {best_score:.4f})")

print(f"\n📁 출력 파일:")
print(f"   • {output_filename} - 예측 결과")
print(f"   • model_info_and_results.json - 모델 정보 및 훈련 결과")
print(f"   • 핵심 컬럼: pred_heat_demand (예측값)")

print(f"\n⏱️ 실행 시간:")
if 'total_time_min' in locals():
    print(f"   🚀 총 훈련 시간: {total_time_min:.1f}분")
    print(f"   ⚡ 평균 모델당: {total_time_min*60/len(models):.1f}초")

print(f"\n🎉 지역난방 열수요 예측 완료!")
print(f"🔬 혁신 포인트: 시즌×브랜치 세분화 + Optuna 자동 최적화")
print(f"📊 총 {len(models)}개 모델로 정밀한 지역별-시즌별 예측 구현")
print("="*80)